In [3]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from src import utils, load_data
from src.models import rigl_maml as rigl
from tqdm.notebook import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torch.utils.tensorboard import SummaryWriter
from collections import namedtuple
from src import utils

from rigl_torch.RigL import RigLScheduler
from    torch import optim

print(f"found gpu: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
found gpu: True


In [4]:
rigl_params = {
    "prune_strategy": {
        "dense_allocation": 0.1,
        "sparsity_distribution": "uniform",
        "delta": 100,
        "alpha": 0.3,
        "grad_accumulation_n": 1,
        "static_topo": False,
        "ignore_linear_layers": False,
        "state_dict": None
    },
    "seeds": {
        "torch": 222,
        "cuda": 222,
        "numpy": 222
    },
    "expr_id": utils.new_expr_id("lth maml"),
    "model_training_params": {
        "training_iterations": 60000, #epoch
        "n_way": 5,                        # number of classes to choose between for each task
        "k_spt": 1,                        # k shot for support set (number of examples per class per task)
        "k_qry": 15,                       # k shot for query set (number of examples per class per task)
        "imgsz": 84,                       # image size
        "imgc": 3,                         # this isn't used anywhere????? no idea what it does???? they say its supposed to be 1 or 3...
        "task_num": 4,                     # meta model batch size
        "meta_lr": 1e-3,                   # meta model learning rate
        "update_lr": 0.01,                 # task specific model learning rate
        "update_step": 5,                  # task specific model training epochs
        "update_step_test": 10,            # task specific model testing epochs
#         "optimizer": ("adam", {"lr": 0.0001}),
#         "loss_func": "cross_entropy",
        "model_name": "MAML",
        "dataset_name": "mini_imagenet",
        "layer_definitions": None
    }
}

rigl_params["model_training_params"]["layer_definitions"] = [
    ('conv2d', [32, 3, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 1, 0]),
    ('flatten', []),
    ('linear', [rigl_params["model_training_params"]["n_way"], 32 * 5 * 5]) # 32 * 5 * 5
]
utils.set_seeds(rigl_params["seeds"])

In [5]:
args = rigl_params["model_training_params"]
dataset = load_data.mini_imagenet(args)
_, _, test_data = dataset
test_accs = []

shuffle DB :train, b:10000, 5-way, 1-shot, 15-query, resize:84
shuffle DB :val, b:100, 5-way, 1-shot, 15-query, resize:84
shuffle DB :test, b:100, 5-way, 1-shot, 15-query, resize:84


In [4]:
for sparsity in [0.1, 0.19, 0.27, 0.34]:
    rigl_params["prune_strategy"]["dense_allocation"] = 1 - sparsity
    accs, best_model, best_model = rigl.run(dataset, rigl_params)
    test_accs.append(rigl.test(best_model, test_data))
    print("Test acc: ", test_accs[-1])

train epoch 0
Time 2020-12-09 10:38:18.608088
 step: 0 	training acc: [0.17       0.21333333 0.21       0.21333333 0.21333333 0.21333333]
validating model...
val acc: [0.2079 0.2354 0.2439 0.244  0.2439 0.2434 0.2446 0.2449 0.2448 0.2445
 0.2448]
 step: 30 	training acc: [0.19666667 0.26666667 0.27       0.27333333 0.27666667 0.27666667]
 step: 60 	training acc: [0.23333333 0.27       0.26333333 0.26       0.25666667 0.25666667]
 step: 90 	training acc: [0.20666667 0.33       0.34       0.37       0.36666667 0.36666667]
 step: 120 	training acc: [0.22333333 0.28666667 0.32333333 0.31333333 0.32333333 0.32333333]
 step: 150 	training acc: [0.18666667 0.37333333 0.38666667 0.38       0.38666667 0.38333333]
 step: 180 	training acc: [0.29       0.39       0.38333333 0.37666667 0.38       0.38      ]
 step: 210 	training acc: [0.18666667 0.25333333 0.27333333 0.27333333 0.27666667 0.27333333]
 step: 240 	training acc: [0.19666667 0.29666667 0.31666667 0.33333333 0.34666667 0.35333333]
 ste

In [5]:
for sparsity in [0.41, 0.47, 0.52, 0.57, 0.61]:
    rigl_params["prune_strategy"]["dense_allocation"] = 1 - sparsity
    accs, best_model, best_model = rigl.run(dataset, rigl_params)
    test_accs.append(rigl.test(best_model, test_data))
    print("Test acc: ", test_accs[-1])

train epoch 0
Time 2020-12-09 16:17:20.122138
 step: 0 	training acc: [0.17333333 0.20666667 0.21       0.20666667 0.20333333 0.20333333]
validating model...
val acc: [0.2046 0.2449 0.2472 0.2482 0.2483 0.248  0.2488 0.2488 0.2499 0.2499
 0.25  ]
 step: 30 	training acc: [0.22       0.32       0.30666667 0.31333333 0.30666667 0.31      ]
 step: 60 	training acc: [0.2        0.29       0.32333333 0.32333333 0.31666667 0.31333333]
 step: 90 	training acc: [0.20333333 0.33333333 0.35       0.35666667 0.36       0.37333333]
 step: 120 	training acc: [0.16666667 0.27       0.29666667 0.29666667 0.29666667 0.29333333]
 step: 150 	training acc: [0.21666667 0.33666667 0.37333333 0.39333333 0.37666667 0.36666667]
 step: 180 	training acc: [0.19       0.31333333 0.33333333 0.33333333 0.33666667 0.33666667]
 step: 210 	training acc: [0.19       0.26666667 0.30333333 0.29333333 0.30333333 0.30333333]
 step: 240 	training acc: [0.22666667 0.30666667 0.32666667 0.34666667 0.36       0.35      ]
 ste

In [ ]:
for sparsity in [0.65, 0.69]:
    rigl_params["prune_strategy"]["dense_allocation"] = 1 - sparsity
    accs, best_model, best_model = rigl.run(dataset, rigl_params)
    test_accs.append(rigl.test(best_model, test_data))
    print("Test acc: ", test_accs[-1])

train epoch 0
Time 2020-12-09 20:45:25.424178
 step: 0 	training acc: [0.22666667 0.3        0.3        0.3        0.30333333 0.30333333]
validating model...
val acc: [0.2014 0.2401 0.2445 0.2457 0.2467 0.2472 0.2477 0.2483 0.2487 0.2494
 0.2498]
 step: 30 	training acc: [0.26       0.32       0.31333333 0.31       0.31       0.31      ]
 step: 60 	training acc: [0.20666667 0.31       0.32       0.31333333 0.31333333 0.30333333]
 step: 90 	training acc: [0.15333333 0.30666667 0.32666667 0.33666667 0.34       0.33      ]
 step: 120 	training acc: [0.18333333 0.30333333 0.27666667 0.30333333 0.29666667 0.29      ]
 step: 150 	training acc: [0.18666667 0.32666667 0.34       0.36       0.35666667 0.36666667]
 step: 180 	training acc: [0.20666667 0.31333333 0.31666667 0.32       0.33333333 0.34      ]
 step: 210 	training acc: [0.17       0.29666667 0.28333333 0.29333333 0.29333333 0.29333333]
 step: 240 	training acc: [0.17       0.29       0.32       0.32666667 0.34333333 0.35333333]
 ste

In [6]:
for sparsity in [0.72, 0.75, 0.77]:
    rigl_params["prune_strategy"]["dense_allocation"] = 1 - sparsity
    accs, best_model, best_model = rigl.run(dataset, rigl_params)
    test_accs.append(rigl.test(best_model, test_data))
    print("Test acc: ", test_accs[-1])

train epoch 0
Time 2020-12-09 22:55:18.756723
 step: 0 	training acc: [0.21666667 0.20666667 0.20666667 0.20333333 0.20333333 0.20333333]
validating model...
val acc: [0.2063 0.2437 0.2417 0.2434 0.2448 0.2454 0.2467 0.2472 0.247  0.2468
 0.2482]
 step: 30 	training acc: [0.21666667 0.28666667 0.29666667 0.29333333 0.3        0.3       ]
 step: 60 	training acc: [0.21333333 0.3        0.27666667 0.27666667 0.27       0.27333333]
 step: 90 	training acc: [0.18333333 0.33       0.35       0.36       0.36666667 0.36      ]
 step: 120 	training acc: [0.24333333 0.31666667 0.33       0.32666667 0.34333333 0.33666667]
 step: 150 	training acc: [0.2        0.35666667 0.38       0.37666667 0.38333333 0.38666667]
 step: 180 	training acc: [0.20333333 0.29       0.33       0.33666667 0.33333333 0.34666667]
 step: 210 	training acc: [0.18666667 0.30333333 0.32       0.32       0.31333333 0.31333333]
 step: 240 	training acc: [0.17333333 0.31333333 0.32666667 0.31666667 0.33       0.33666667]
 ste